# Autoencoder
- Auto = self
- encode = convert into a different form
- Autoencoder = a system that teaches itself how to encode information
- Number outputs correspond to the entry from the model

# Structure of the layers
- Encode: input > layers > bottleneck or latent code (central node)

#  Goal of autoencoder:
- Get the output to match the input closer possible
- data compression of dimension reduction
- data cleaning (denoising, despeckling, occlusion)  
- feature extraction
- anomaly / fraud detection
- pretraining deep or complex models   


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [3]:
from preprocess.preprocess import MovieLens
ml = MovieLens()

In [6]:
matrix = ml.get_matrix().to_numpy()

In [13]:
# matrix[:,]